# Optimize Storage (Parallel Copy to SSD)


In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import multiprocessing


SOURCE_DIR = "/data"          
DEST_DIR = "/tmp/dataset"     


num_cores = multiprocessing.cpu_count()
MAX_WORKERS = num_cores * 4   
print(f"Detected {num_cores} CPU cores. Using {MAX_WORKERS} threads for copying.")


def copy_file(src, dst):
    try:
        shutil.copy2(src, dst)
    except Exception as e:
        print(f"Error: {e}")

print("Scanning volume and creating folder structure...")
files_to_copy = []

if not os.path.exists(SOURCE_DIR):
    print("Error: Volume not found at /data")
else:
    for root, dirs, files in os.walk(SOURCE_DIR):
        if ".ipynb_checkpoints" in root: continue
        rel_path = os.path.relpath(root, SOURCE_DIR)
        dest_path = os.path.join(DEST_DIR, rel_path)
        os.makedirs(dest_path, exist_ok=True)
        
        for file in files:
            files_to_copy.append((os.path.join(root, file), os.path.join(dest_path, file)))

    print(f"Found {len(files_to_copy)} files. Starting Transfer...")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        list(tqdm(executor.map(lambda p: copy_file(*p), files_to_copy), total=len(files_to_copy)))

    print("\nDATA TRANSFER COMPLETE! Using Fast Local Storage (/tmp/dataset)")

🚀 Detected 17 CPU cores. Using 68 threads for copying.
Scanning volume and creating folder structure...
Found 13454 files. Starting Transfer...


100%|██████████| 13454/13454 [01:13<00:00, 182.78it/s]



DATA TRANSFER COMPLETE! Using Fast Local Storage (/tmp/dataset)


# Imports & GPU Verification


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

DATA_ROOT = "/tmp/dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR = os.path.join(DATA_ROOT, "val")

if not os.path.exists(TRAIN_DIR):
    TRAIN_DIR = DATA_ROOT
    VAL_DIR = DATA_ROOT
    print(f"'train' folder not found. Using root: {TRAIN_DIR}")
else:
    print(f"Training Data Path: {TRAIN_DIR}")

2025-12-30 07:48:20.423114: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-30 07:48:20.510018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-30 07:48:24.450748: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/local/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: Futur

TensorFlow Version: 2.20.0
Num GPUs Available:  1
Training Data Path: /tmp/dataset/train


# Fast Data Generator


In [ ]:
class VideoGenerator(Sequence):
    def __init__(self, data_dir, classes, batch_size, img_size, seq_len):
        self.data_dir = data_dir
        self.classes = classes
        self.batch_size = batch_size
        self.img_size = img_size
        self.seq_len = seq_len
        self.class_map = {cls: i for i, cls in enumerate(classes)}
        self.num_classes = len(classes)
        self.files = []
        self.labels = []
        
        for cls in self.classes:
            cls_path = os.path.join(self.data_dir, cls)
            if os.path.isdir(cls_path):
                for vid in os.listdir(cls_path):
                    self.files.append(os.path.join(cls_path, vid))
                    self.labels.append(self.class_map[cls])
        
        if len(self.files) > 0:
            combined = list(zip(self.files, self.labels))
            np.random.shuffle(combined)
            self.files[:], self.labels[:] = zip(*combined)
        else:
            print(f"Warning: No files found in {data_dir}")

    def __len__(self):
        return int(np.ceil(len(self.files) / self.batch_size))

    def __getitem__(self, idx):
        batch_x_paths = self.files[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y_labels = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]
        
        X = []
        for vid_path in batch_x_paths:
            frames = self.process_video(vid_path)
            X.append(frames)
        
        return np.array(X), to_categorical(batch_y_labels, num_classes=self.num_classes)

    def process_video(self, path):
        cap = cv2.VideoCapture(path)
        frames = []
        try:
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            skip = max(int(total_frames / self.seq_len), 1)
            
            for i in range(self.seq_len):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i * skip)
                ret, frame = cap.read()
                if not ret: break
                frame = cv2.resize(frame, (self.img_size, self.img_size))
                frames.append(frame)
        finally:
            cap.release()
            
        while len(frames) < self.seq_len:
            frames.append(np.zeros((self.img_size, self.img_size, 3)))
            
        return np.array(frames[:self.seq_len]) / 255.0

print("Generator Class Ready.")

Generator Class Ready.


# Build CNN + LSTM Model


In [ ]:

IMG_SIZE = 128
SEQ_LENGTH = 16
BATCH_SIZE = 32
EPOCHS = 15
classes = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
print(f"Classes Found ({len(classes)}): {classes[:5]}...")

train_gen = VideoGenerator(TRAIN_DIR, classes, BATCH_SIZE, IMG_SIZE, SEQ_LENGTH)
val_gen = VideoGenerator(VAL_DIR, classes, BATCH_SIZE, IMG_SIZE, SEQ_LENGTH)

def build_model():
    cnn = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    cnn.trainable = False  
    
    model = Sequential()
    model.add(TimeDistributed(cnn, input_shape=(SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3)))
    model.add(TimeDistributed(GlobalAveragePooling2D()))
    
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    
    model.add(Dense(len(classes), activation='softmax'))
    
    return model

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Classes Found (101): ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam']...


I0000 00:00:1767080914.797223      21 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20817 MB memory:  -> device: 0, name: NVIDIA A10, pci bus id: 0000:ca:00.0, compute capability: 8.6


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 16, 4, 4, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 16, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │       344,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 101)            │         6,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,608,869 (9.95 MB)

 Trainable params: 350,885 (1.34 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Train with Multiprocessing


In [ ]:
import tensorflow as tf
import multiprocessing


print("Setting up High-Speed Data Pipeline...")

def load_video_wrapper(path_tensor, label_tensor):
    path = path_tensor.numpy().decode('utf-8')
    label = label_tensor.numpy()
    
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        skip = max(int(total_frames / SEQ_LENGTH), 1)
        for i in range(SEQ_LENGTH):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i * skip)
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame.astype(np.float32) / 255.0
            frames.append(frame)
    finally:
        cap.release()
        
    while len(frames) < SEQ_LENGTH:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.float32))
        
    return np.array(frames[:SEQ_LENGTH]), label

def process_path(path, label):
    video, label = tf.py_function(
        func=load_video_wrapper,
        inp=[path, label],
        Tout=[tf.float32, tf.float32]
    )
    video.set_shape((SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3))
    label.set_shape((len(classes),))
    return video, label

def create_dataset(generator):
    paths = generator.files
    labels = to_categorical(generator.labels, num_classes=len(classes))
    
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE) 
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

print("Building Parallel Pipelines...")
train_ds = create_dataset(train_gen)
val_ds = create_dataset(val_gen)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/data/best_model_checkpoint.h5", 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max',
    verbose=1
)

print("STARTING TRAINING ")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

Setting up High-Speed Data Pipeline...
Building Parallel Pipelines...
STARTING TRAINING 
Epoch 1/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 915ms/step - accuracy: 0.1396 - loss: 3.8562
Epoch 1: val_accuracy improved from None to 0.43335, saving model to /data/best_model_checkpoint.h5



Epoch 1: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.1850 - loss: 3.6189 - val_accuracy: 0.4334 - val_loss: 2.9195
Epoch 2/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.3065 - loss: 2.9731
Epoch 2: val_accuracy improved from 0.43335 to 0.53377, saving model to /data/best_model_checkpoint.h5



Epoch 2: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 274s 867ms/step - accuracy: 0.3359 - loss: 2.8375 - val_accuracy: 0.5338 - val_loss: 2.3637
Epoch 3/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 794ms/step - accuracy: 0.4072 - loss: 2.4633
Epoch 3: val_accuracy improved from 0.53377 to 0.59474, saving model to /data/best_model_checkpoint.h5



Epoch 3: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 306s 971ms/step - accuracy: 0.4295 - loss: 2.3575 - val_accuracy: 0.5947 - val_loss: 1.9773
Epoch 4/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 910ms/step - accuracy: 0.4787 - loss: 2.1587
Epoch 4: val_accuracy improved from 0.59474 to 0.64136, saving model to /data/best_model_checkpoint.h5



Epoch 4: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 340s 1s/step - accuracy: 0.4959 - loss: 2.0770 - val_accuracy: 0.6414 - val_loss: 1.7406
Epoch 5/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5334 - loss: 1.8843
Epoch 5: val_accuracy improved from 0.64136 to 0.67723, saving model to /data/best_model_checkpoint.h5


...


Epoch 10: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 410s 1s/step - accuracy: 0.7103 - loss: 1.1085 - val_accuracy: 0.7806 - val_loss: 0.9132
Epoch 11/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7326 - loss: 1.0291
Epoch 11: val_accuracy improved from 0.78063 to 0.79558, saving model to /data/best_model_checkpoint.h5



Epoch 11: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 474s 2s/step - accuracy: 0.7398 - loss: 1.0052 - val_accuracy: 0.7956 - val_loss: 0.8952
Epoch 12/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7444 - loss: 0.9760
Epoch 12: val_accuracy improved from 0.79558 to 0.80275, saving model to /data/best_model_checkpoint.h5



Epoch 12: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 495s 2s/step - accuracy: 0.7541 - loss: 0.9425 - val_accuracy: 0.8027 - val_loss: 0.8153
Epoch 13/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7585 - loss: 0.8891
Epoch 13: val_accuracy improved from 0.80275 to 0.80454, saving model to /data/best_model_checkpoint.h5



Epoch 13: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 464s 1s/step - accuracy: 0.7695 - loss: 0.8620 - val_accuracy: 0.8045 - val_loss: 0.7798
Epoch 14/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7717 - loss: 0.8430
Epoch 14: val_accuracy improved from 0.80454 to 0.81470, saving model to /data/best_model_checkpoint.h5



Epoch 14: finished saving model to /data/best_model_checkpoint.h5
315/315 ━━━━━━━━━━━━━━━━━━━━ 454s 1s/step - accuracy: 0.7847 - loss: 0.8022 - val_accuracy: 0.8147 - val_loss: 0.7426
Epoch 15/15
315/315 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7871 - loss: 0.7982
Epoch 15: val_accuracy did not improve from 0.81470
315/315 ━━━━━━━━━━━━━━━━━━━━ 409s 1s/step - accuracy: 0.7924 - loss: 0.7846 - val_accuracy: 0.8027 - val_loss: 0.7386


# Save Final Results


In [ ]:

save_path = "/data/final_model_cnn_lstm.h5"
model.save(save_path)

with open("/data/classes.txt", "w") as f:
    f.write("\n".join(classes))

print(f"Success! Model saved to: {save_path}")
print("You can now stop the Modal app and download the model.")

Success! Model saved to: /data/final_model_cnn_lstm.h5
You can now stop the Modal app and download the model.
